# Entrenamiento del modelo de predicción de irradiación solar

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [8]:
df = pd.read_csv('datos_v2.csv', delimiter=';')
# df2 = pd.read_csv('datos.csv', delimiter=';')

In [9]:
# Renombrar columnas
df.rename(columns={'Temperatura de módulo | Sensor Card / Box (1)': 'temperaturaSensor'}, inplace=True)
df.rename(columns={'Temperatura ambiente | Sensor Card / Box (1)': 'temperaturaAmbiente'}, inplace=True)
df.rename(columns={'Irradiación | Sensor Card / Box (1)': 'irradiacion'}, inplace=True)
df.rename(columns={'Fecha y hora': 'fecha'}, inplace=True)

# Quedarme con dos columnas de entrada y una de salida(irradiacion)
df = df[['fecha','temperaturaAmbiente','temperaturaSensor','irradiacion']]

# Eliminar los valores nulos
df.dropna(inplace=True)

df['irradiacion'] = df['irradiacion'].apply(lambda x: float(str(x).strip().replace(',','.')))

# Solo quedarme con los valores que hay irradiación (Día)
df = df[df['irradiacion'] > 0]
df


,fecha,temperaturaAmbiente,temperaturaSensor,irradiacion
0,01.03.2024 15:45,"24,00","31,00",288.00
1,01.03.2024 15:50,"24,00","34,00",293.62
2,01.03.2024 15:55,"24,00","34,00",340.00
3,01.03.2024 16:00,"24,00","36,00",469.00
4,01.03.2024 16:05,"24,00","36,00",385.00
...,...,...,...,...
12395,26.04.2024 09:20,"23,00","33,00",302.00
12396,26.04.2024 09:30,"23,00","34,00",303.00
12397,26.04.2024 09:40,"23,00","32,99",264.03
12398,26.04.2024 09:50,"23,00","32,00",268.00


In [10]:
# Transformar la fecha a datetime
df['fecha'] = pd.to_datetime(df['fecha'], format='%d.%m.%Y %H:%M')
df['hora'] = [i.hour for i in df['fecha']]
df['minuto'] = [i.minute for i in df['fecha']]
# Parsear las columnas temperaturas a float
df['temperaturaAmbiente'] = df['temperaturaAmbiente'].apply(lambda x: float(str(x).strip().replace(',','.')))
df['temperaturaSensor'] = df['temperaturaSensor'].apply(lambda x: float(str(x).strip().replace(',','.')))
df['tiempo'] = df.apply(lambda row: row['hora'] + (row['minuto'] // 5) * 0.083, axis=1)

In [12]:
# Separar las características (X) y la variable objetivo (y)
X_modulo = df[['temperaturaAmbiente','tiempo']]
y_modulo = df['temperaturaSensor']

X_irradiacion = df[['temperaturaSensor', 'temperaturaAmbiente', 'tiempo']]
y_irradiacion = df['irradiacion']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_modulo_train, X_modulo_test, y_modulo_train, y_modulo_test = train_test_split(X_modulo, y_modulo, test_size=0.2, random_state=42)
X_irradiacion_train, X_irradiacion_test, y_irradiacion_train, y_irradiacion_test = train_test_split(X_irradiacion, y_irradiacion, test_size=0.2, random_state=42)
# Crear un modelo de regresión lineal
model_modulo = LinearRegression()
model_irradiacion = LinearRegression()

# Entrenar el modelo
model_modulo.fit(X_modulo_train, y_modulo_train)
model_irradiacion.fit(X_irradiacion_train, y_irradiacion_train)


LinearRegression()

In [13]:
# Realizar predicciones en el conjunto de prueba
y_modulo_pred = model_modulo.predict(X_modulo_test)
y_irradiacion_pred = model_irradiacion.predict(X_irradiacion_test)

# Evaluar el rendimiento del modelo
r2_modulo = r2_score(y_modulo_test, y_modulo_pred)
r2_irradiacion = r2_score(y_irradiacion_test, y_irradiacion_pred)

# Imprimir los resultados
# print("Coeficientes del modelo:", model.coef_)
# print("Intercepción del modelo:", model.intercept_)
# print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación Módulo (R²):", r2_modulo)
print("Coeficiente de determinación Irradiación (R²):", r2_irradiacion)


Coeficiente de determinación Módulo (R²): 0.783304227554866
Coeficiente de determinación Irradiación (R²): 0.8650489145304878


In [14]:
# En la prediccion se debe pasar una lista con la hora
a = model_modulo.predict([[23, 14+(40//5)*0.083]])
irradiacion = model_irradiacion.predict([[a[0], 23, 14+(40//5)*0.083]])
irradiacion[0]

[350.8963385]


c:\Users\luisj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\luisj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


### Guardar el modelo

In [15]:
import joblib

joblib.dump(model_modulo, 'modelo_regresion_lineal_modulo.joblib')
joblib.dump(model_irradiacion, 'modelo_regresion_lineal_irradiacion.joblib')

['modelo_regresion_lineal_irradiacion.joblib']

In [16]:
# Cargar el modelo
model_modulo = joblib.load('modelo_regresion_lineal_modulo.joblib')
model_irradiacion = joblib.load('modelo_regresion_lineal_irradiacion.joblib')

In [17]:
a = model_modulo.predict([[23, 14+(40//5)*0.083]])
irradiacion = model_irradiacion.predict([[a[0], 23, 14+(40//5)*0.083]])
irradiacion[0]

c:\Users\luisj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\luisj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


350.8963385022613